# Get Hardees locations

#### Load Python tools and Jupyter config

In [1]:
%load_ext lab_black

In [23]:
import json
import requests
import warnings
import pandas as pd
import altair as alt
from time import sleep
import geopandas as gpd
from random import randint
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = None
warnings.filterwarnings("ignore")

In [4]:
place = "hardees"

## Read data

#### Get populous ZIP Codes and their lon/lat

In [5]:
zips_src = (
    gpd.read_file("../_reference/data/zips_centroids.geojson")[
        ["ID", "ST_ABBREV", "TOTPOP_CY", "geometry"]
    ]
    .sort_values("TOTPOP_CY", ascending=False)
    .reset_index(drop=True)
)
zips_src.columns = zips_src.columns.str.lower()

In [6]:
zips_src["longitude"] = zips_src.centroid.x
zips_src["latitude"] = zips_src.centroid.y

#### Limit to states where Hardees doesn't operate

In [7]:
exclude = [
    "CO",
    "DC",
    "WA",
    "OR",
    "NJ",
    "AZ",
    "ME",
    "RI",
    "CA",
    "NM",
    "CT",
    "ID",
    "NH",
    "HI",
    "AK",
    "TX",
    "MA",
    "NV",
    "VT",
]

In [8]:
zips = zips_src[~zips_src["st_abbrev"].isin(exclude)].copy()

In [9]:
zips.head()

,id,st_abbrev,totpop_cy,geometry,longitude,latitude
2,60629,IL,117919,POINT (-87.71233 41.77497),-87.712328,41.774969
3,11368,NY,117876,POINT (-73.85562 40.74959),-73.855624,40.749594
9,11226,NY,108846,POINT (-73.95732 40.64664),-73.957322,40.646640
11,11373,NY,106655,POINT (-73.87846 40.73885),-73.878464,40.738850
13,11385,NY,104889,POINT (-73.88965 40.70124),-73.889652,40.701242


---

#### Set up requests

In [10]:
import requests

cookies = {
    "_gcl_au": "1.1.516157714.1706369182",
    "OptanonConsent": "isGpcEnabled=0&datestamp=Sat+Jan+27+2024+07%3A26%3A21+GMT-0800+(Pacific+Standard+Time)&version=202301.1.0&isIABGlobal=false&hosts=&consentId=e826d9d7-168a-46eb-be52-07b4bfff3bdf&interactionCount=0&landingPath=https%3A%2F%2Fwww.hardees.com%2Flocations&groups=C0003%3A1%2CC0004%3A1%2CC0002%3A1%2CC0001%3A1",
    "_rdt_uuid": "1706369181864.ccf9a890-65b3-43e2-93b2-fcc91f0ecb69",
    "_scid": "974037f8-067a-4136-936c-17b08ded25c5",
    "_scid_r": "974037f8-067a-4136-936c-17b08ded25c5",
    "_derived_epik": "dj0yJnU9NW8wc1EzeDFJWUVVbmY5b0NIRmdRZEJpNGY5bVRCdjQmbj1hZUpXdXJfS01oNHpwc3BYV0xVb2hRJm09MSZ0PUFBQUFBR1cxSUo0JnJtPTEmcnQ9QUFBQUFHVzFJSjQmc3A9Mg",
    "_pin_unauth": "dWlkPU5UQXdaRGd3WlRNdE5qRTVNeTAwWWpkaUxUazNPVGt0T0dKak0yTmxOR014WkdZeA",
    "_fbp": "fb.1.1706369182427.574499949",
    "_gid": "GA1.2.2137922063.1706369182",
    "_dc_gtm_UA-168018701-6": "1",
    "_tq_id.TV-7209093654-1.221e": "243b8ae9a1246d2e.1706369182.0.1706369182..",
    "_tt_enable_cookie": "1",
    "_ttp": "70fl9xnW-KVCgGwM-jDR3L1nnR8",
    "_ga_LHYVJB2PVQ": "GS1.2.1706369182.1.0.1706369182.0.0.0",
    "_ga": "GA1.1.354940790.1706369182",
    "_ga_2Y3L4YDWMY": "GS1.1.1706369182.1.0.1706369183.0.0.0",
    "_sctr": "1%7C1706342400000",
}

headers = {
    "authority": "www.hardees.com",
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "referer": "https://www.hardees.com/locations",
    "sec-ch-ua": '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "x-requested-with": "XMLHttpRequest",
}

#### Loop through the list to set a search radius in each ZIP

In [11]:
restaurant_list = []

for k, v in tqdm(zips.head(1000).iterrows()):

    params = {
        "Latitude": v["latitude"],
        "Longitude": v["longitude"],
        "Radius": "100",
    }

    response = requests.get(
        "https://www.hardees.com/api/locationapi",
        params=params,
        cookies=cookies,
        headers=headers,
    )

    data = pd.DataFrame(response.json()["locations"])
    restaurant_list.append(data)

0it [00:00, ?it/s]

#### Dataframe from list

In [12]:
src = pd.concat(restaurant_list)

#### Clean up

In [13]:
src["id"] = (
    src["fullDisplayAddress"]
    .str.split(" - ", expand=True)[0]
    .str.replace("Restaurant ", "")
    .str.strip()
    .astype(str)
)

src["zipCode"] = src["zipCode"].str[:5]

#### Just the columns we need

In [14]:
df = (
    src[
        [
            "id",
            "address",
            "city",
            "state",
            "zipCode",
            "phoneNumber",
            "latitude",
            "longitude",
        ]
    ]
    .drop_duplicates()
    .copy()
    .rename(columns={"zipCode": "zip", "phoneNumber": "phone"})
)

In [15]:
df.head()

,id,address,city,state,zip,phone,latitude,longitude
0,1505938,1533 E 162ND STREET,SOUTH HOLLAND,IL,60473,(708) 331-7399,41.601400,-87.579600
1,1506621,ST. RT 6 & BRISBIN RD,MORRIS,IL,60450,(815) 705-9259,41.415710,-88.365560
2,1506491,5223 FRANKLIN STREET,MICHIGAN CITY,IN,46360,(219) 268-1010,41.672378,-86.894499
3,1505746,12 W NORTHBROOK DR,DWIGHT,IL,60420,(815) 584-9596,41.113495,-88.415228
4,1503201,2625 COLUMBUS ST,OTTAWA,IL,61350,(815) 434-8688,41.369520,-88.836520


In [16]:
df.tail()

,id,address,city,state,zip,phone,latitude,longitude
17,1504047,711 WOOD AVE E,BIG STONE GAP,VA,24219,(276) 523-4704,36.870110,-82.771860
12,1501429,1113 E 5TH AVE,FLORALA,AL,36442,(334) 858-3711,31.002804,-86.324225
17,1501353,1202 E BYPASS,ANDALUSIA,AL,36420,(334) 222-7315,31.319240,-86.460575
18,1501384,803 FLORALA HWY,OPP,AL,36467,(334) 493-3314,31.273591,-86.248597
19,1501391,845 LIBERTY HILL DR,EVERGREEN,AL,36401,(251) 578-1983,31.454496,-86.962969


In [17]:
len(df.drop_duplicates())

1419

---

## Geography

#### Make a geodataframe from lon/lat

In [18]:
df_geo = df.copy()

In [19]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
).set_crs("epsg:4326")

---

## Maps

#### US states background

In [24]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place.title()} locations")
    .project("albersUsa")
)

#### Location points map

In [25]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=10, color="red")
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [26]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value("red"),
        tooltip=["state:N", "count:Q"],
    )
    .properties(title=f"Number of {place} in US, by average lon/lat of locations")
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### CSV

In [27]:
df.to_csv(f"data/processed/{place.lower()}_locations.csv", index=False)

#### JSON

In [28]:
df.to_json(f"data/processed/{place.lower()}_locations.json", indent=4, orient="records")

#### GeoJSON

In [29]:
col = "tags"
gdf.loc[:, gdf.columns != col].to_file(
    f"data/processed/{place.lower()}_locations.geojson", driver="GeoJSON"
)